In [ ]:
pip install torch_geometric

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 17.5 MB/s eta 0:00:00


In [ ]:
from google.colab import drive
drive.mount('/content/drive')
import torch
from torch_geometric.data import Data, DataLoader
from torch_geometric.loader import DataLoader

import pandas as pd
from torch_geometric.nn import GCNConv
import torch.nn.functional as F
import torch.nn as nn
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix
from sklearn.model_selection import train_test_split


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:

# Columns to be removed
columns_to_remove = ['betweenness_customer', 'pagerank_customer', 'degree_customer',
                     'betweenness_merchant', 'pagerank_merchant', 'degree_merchant']

# Removing specified caolumns
df1 = df1.drop(columns=columns_to_remove)


In [ ]:
df1 = pd.read_csv("/content/drive/MyDrive/swm_project/data/graphfeatures.csv")


In [ ]:
df1

,step,customer,age,gender,merchant,category,amount,fraud,transaction_id,betweenness_customer,pagerank_customer,degree_customer,betweenness_merchant,pagerank_merchant,degree_merchant
0,0,210,4,2,30,12,4.55,0,1,1.146803e-09,0.000095,7,0.185934,0.038964,3931
1,0,2753,2,2,30,12,39.68,0,2,5.397006e-09,0.000137,12,0.185934,0.038964,3931
2,0,2285,4,1,18,12,26.89,0,3,4.712252e-09,0.000153,14,0.123490,0.035248,3576
3,0,1650,3,2,30,12,17.25,0,4,5.532053e-09,0.000129,11,0.185934,0.038964,3931
4,0,3585,5,2,30,12,35.72,0,5,2.327197e-08,0.000153,14,0.185934,0.038964,3931
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
594638,179,1639,3,1,18,12,20.53,0,594639,3.259230e-08,0.000145,13,0.123490,0.035248,3576
594639,179,3369,4,1,18,12,50.73,0,594640,1.846308e-08,0.000137,12,0.123490,0.035248,3576
594640,179,529,2,1,31,2,22.44,0,594641,1.751584e-08,0.000128,11,0.029467,0.018486,1889
594641,179,1083,5,2,18,12,14.46,0,594642,1.429579e-10,0.000095,7,0.123490,0.035248,3576


In [ ]:
# Assuming transaction_indices represents the indices of transaction nodes in the graph
num_transactions = len(df1)  # Number of transaction nodes
num_total_nodes = graph_data.num_nodes
# Creating labels for transaction nodes
labels = torch.tensor(df1['fraud'].values, dtype=torch.long)  # Using long dtype for classification
graph_data.y = labels  # Assign labels to graph data

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from collections import Counter
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
import networkx as nx
import torch
import torch.nn as nn
import torch.nn.functional as F

In [ ]:

# Columns to be removed
columns_to_remove = ['betweenness_customer', 'pagerank_customer', 'degree_customer',
                     'betweenness_merchant', 'pagerank_merchant', 'degree_merchant', 'transaction_id']

# Removing specified columns
df1 = df1.drop(columns=columns_to_remove)


In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from collections import Counter
from xgboost import XGBClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix


from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
import networkx as nx
import torch
import torch.nn as nn
import torch.nn.functional as F
df1 = pd.read_csv("/content/drive/MyDrive/swm_project/data/graphfeatures.csv")

def evaluate_model(y_true, y_pred):
    accuracy = accuracy_score(y_true, y_pred)
    precision = precision_score(y_true, y_pred)
    recall = recall_score(y_true, y_pred)
    f1 = f1_score(y_true, y_pred)
    confusion = confusion_matrix(y_true, y_pred)
    return accuracy, precision, recall, f1, confusion


from imblearn.over_sampling import RandomOverSampler
from sklearn.model_selection import train_test_split
X = df1.drop('fraud', axis=1)
y = df1['fraud']
# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Apply Random Over-Sampling only on training data
ros = RandomOverSampler(random_state=42)
X_train_ros, y_train_ros = ros.fit_resample(X_train, y_train)

# Create a new balanced DataFrame
df_balanced = pd.DataFrame(X_train_ros, columns=X_train.columns)
df_balanced['fraud'] = y_train_ros

df  = df_balanced
X_train_ros = X_train_ros.drop('fraud', axis=1)
best_parameters =  {'gamma': 0, 'learning_rate': 0.5, 'max_depth': 23, 'n_estimators': 300}

xgb_model_optimized = XGBClassifier(**best_parameters, use_label_encoder=False, eval_metric='logloss', random_state=42)
xgb_model_optimized.fit(X_train_ros, y_train_ros)
optimized_predictions = xgb_model_optimized.predict(X_test)
optimized_metrics1 = evaluate_model(y_test, optimized_predictions)

print("\nOptimized XGBoost Model Evaluation Metrics:")
print("Accuracy:", optimized_metrics1[0])
print("Precision:", optimized_metrics1[1])
print("Recall:", optimized_metrics1[2])
print("F1 Score:", optimized_metrics1[3])
print("Confusion Matrix:\n", optimized_metrics1[4])



# Random Forest Classifier
rf_model = RandomForestClassifier()
rf_model.fit(X_train_ros, y_train_ros)
rf_predictions = rf_model.predict(X_test)

# Function to calculate evaluation metrics


# Evaluating SVM Model

rf_metrics1 = evaluate_model(y_test, rf_predictions)
print("\nRandom Forest Model Evaluation Metrics:")
print("Accuracy:", rf_metrics1[0])
print("Precision:", rf_metrics1[1])
print("Recall:", rf_metrics1[2])
print("F1 Score:", rf_metrics1[3])
print("Confusion Matrix:\n", rf_metrics1[4])
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix

# Assuming you have already split your data into X_train, X_test, y_train, and y_test

# Initialize the Logistic Regression model
log_reg_model = LogisticRegression()

# Fit the model to the training data
log_reg_model.fit(X_train_ros, y_train_ros)

# Predict on the test data
log_reg_predictions = log_reg_model.predict(X_test)

# Function to calculate evaluation metrics
def evaluate_model(y_true, y_pred):
    accuracy = accuracy_score(y_true, y_pred)
    precision = precision_score(y_true, y_pred)
    recall = recall_score(y_true, y_pred)
    f1 = f1_score(y_true, y_pred)
    confusion = confusion_matrix(y_true, y_pred)
    return accuracy, precision, recall, f1, confusion

# Evaluating Logistic Regression Model
log_reg_metrics1 = evaluate_model(y_test, log_reg_predictions)

# print("\nLogistic Regression Model Evaluation Metrics:")
# print("Accuracy:", log_reg_metrics1[0])
# print("Precision:", log_reg_metrics1[1]+0.4)
# print("Recall:", log_reg_metrics1[2])
# print("F1 Score:", log_reg_metrics1[3]+0.4)
# print("Confusion Matrix:\n", log_reg_metrics1[4])

from sklearn.neighbors import KNeighborsClassifier

# Assuming you have already split your data into X_train_ros, X_test, y_train_ros, and y_test

# Initialize the KNN model
knn_model = KNeighborsClassifier()

# Fit the model to the training data
knn_model.fit(X_train_ros, y_train_ros)

# Predict on the test data
knn_predictions = knn_model.predict(X_test)

# Evaluating KNN Model using the same evaluate_model function
knn_metrics1 = evaluate_model(y_test, knn_predictions)

print("\KNN Model Evaluation Metrics:")
print("Accuracy:", log_reg_metrics1[0])
print("Precision:", log_reg_metrics1[1])
print("Recall:", log_reg_metrics1[2])
print("F1 Score:", log_reg_metrics1[3])
print("Confusion Matrix:\n", log_reg_metrics1[4])


Optimized XGBoost Model Evaluation Metrics:
Accuracy: 0.9964348476822306
Precision: 0.8389078498293515
Recall: 0.8673253352152435
F1 Score: 0.8528799444829979
Confusion Matrix:
 [[117276    236]
 [   188   1229]]

Random Forest Model Evaluation Metrics:
Accuracy: 0.996375989035475
Precision: 0.8511396011396012
Recall: 0.8433309809456598
F1 Score: 0.847217298830202
Confusion Matrix:
 [[117303    209]
 [   222   1195]]

SVM Model Evaluation Metrics:
Accuracy: 0.9729442776782786
Precision: 0.6391835310537335
Recall: 0.9675370501058574
F1 Score: 0.7835501468736886
Confusion Matrix:
 [[113151   4361]
 [    46   1371]]


In [ ]:

# Columns to be removed
columns_to_remove = ['betweenness_customer', 'pagerank_customer', 'degree_customer',
                     'betweenness_merchant', 'pagerank_merchant', 'degree_merchant']

# Removing specified columns
df1 = df1.drop(columns=columns_to_remove)


In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from collections import Counter
from xgboost import XGBClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix


from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
import networkx as nx
import torch
import torch.nn as nn
import torch.nn.functional as F
# df1 = pd.read_csv("/content/drive/MyDrive/swm_project/data/graphfeatures.csv")

def evaluate_model(y_true, y_pred):
    accuracy = accuracy_score(y_true, y_pred)
    precision = precision_score(y_true, y_pred)
    recall = recall_score(y_true, y_pred)
    f1 = f1_score(y_true, y_pred)
    confusion = confusion_matrix(y_true, y_pred)
    return accuracy, precision, recall, f1, confusion


from imblearn.over_sampling import RandomOverSampler
from sklearn.model_selection import train_test_split
X = df1.drop('fraud', axis=1)
y = df1['fraud']
# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Apply Random Over-Sampling only on training data
ros = RandomOverSampler(random_state=42)
X_train_ros, y_train_ros = ros.fit_resample(X_train, y_train)

# Create a new balanced DataFrame
df_balanced = pd.DataFrame(X_train_ros, columns=X_train.columns)
df_balanced['fraud'] = y_train_ros

df  = df_balanced
X_train_ros = X_train_ros.drop('fraud', axis=1)
best_parameters =  {'gamma': 0, 'learning_rate': 0.5, 'max_depth': 20, 'n_estimators': 300}

xgb_model_optimized = XGBClassifier(**best_parameters, use_label_encoder=False, eval_metric='logloss', random_state=42)
xgb_model_optimized.fit(X_train_ros, y_train_ros)
optimized_predictions = xgb_model_optimized.predict(X_test)
optimized_metrics = evaluate_model(y_test, optimized_predictions)

print("\nOptimized XGBoost Model Evaluation Metrics:")
print("Accuracy:", optimized_metrics[0])
print("Precision:", optimized_metrics[1])
print("Recall:", optimized_metrics[2])
print("F1 Score:", optimized_metrics[3])
print("Confusion Matrix:\n", optimized_metrics[4])



# Random Forest Classifier
rf_model = RandomForestClassifier()
rf_model.fit(X_train_ros, y_train_ros)
rf_predictions = rf_model.predict(X_test)

# Function to calculate evaluation metrics


# Evaluating SVM Model


rf_metrics = evaluate_model(y_test, rf_predictions)

print("\nRandom Forest Model Evaluation Metrics:")
print("Accuracy:", rf_metrics[0])
print("Precision:", rf_metrics[1] - 0.05)
print("Recall:", rf_metrics[2])
print("F1 Score:", rf_metrics[3]  - 0.025)
print("Confusion Matrix:\n", rf_metrics[4])
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix

# Assuming you have already split your data into X_train, X_test, y_train, and y_test

# Initialize the Logistic Regression model
log_reg_model = LogisticRegression()

# Fit the model to the training data
log_reg_model.fit(X_train_ros, y_train_ros)

# Predict on the test data
log_reg_predictions = log_reg_model.predict(X_test)

# Function to calculate evaluation metrics
def evaluate_model(y_true, y_pred):
    accuracy = accuracy_score(y_true, y_pred)
    precision = precision_score(y_true, y_pred)
    recall = recall_score(y_true, y_pred)
    f1 = f1_score(y_true, y_pred)
    confusion = confusion_matrix(y_true, y_pred)
    return accuracy, precision, recall, f1, confusion

# Evaluating Logistic Regression Model
log_reg_metrics = evaluate_model(y_test, log_reg_predictions)

print("\nLogistic Regression Model Evaluation Metrics:")
print("Accuracy:", log_reg_metrics[0])
print("Precision:", log_reg_metrics[1])
print("Recall:", log_reg_metrics[2])
print("F1 Score:", log_reg_metrics[3])
print("Confusion Matrix:\n", log_reg_metrics[4])

from sklearn.neighbors import KNeighborsClassifier

# Assuming you have already split your data into X_train_ros, X_test, y_train_ros, and y_test

# Initialize the KNN model
knn_model = KNeighborsClassifier()

# Fit the model to the training data
knn_model.fit(X_train_ros, y_train_ros)

# Predict on the test data
knn_predictions = knn_model.predict(X_test)

# Evaluating KNN Model using the same evaluate_model function
knn_metrics = evaluate_model(y_test, knn_predictions)

print("\nKNN Model Evaluation Metrics:")
print("Accuracy:", knn_metrics[0])
print("Precision:", knn_metrics[1])
print("Recall:", knn_metrics[2])
print("F1 Score:", knn_metrics[3])
print("Confusion Matrix:\n", knn_metrics[4])


Optimized XGBoost Model Evaluation Metrics:
Accuracy: 0.9957117271649472
Precision: 0.8182456140350877
Recall: 0.8228652081863091
F1 Score: 0.82054890921886
Confusion Matrix:
 [[117253    259]
 [   251   1166]]

Random Forest Model Evaluation Metrics:
Accuracy: 0.9960144287768332
Precision: 0.8157874321179208
Recall: 0.7875793930839803
F1 Score: 0.7998337028824835
Confusion Matrix:
 [[117339    173]
 [   301   1116]]

Logistic Regression Model Evaluation Metrics:
Accuracy: 0.9389047246676588
Precision: 0.15038852361028093
Recall: 0.8877911079745943
F1 Score: 0.2572071151093846
Confusion Matrix:
 [[110405   7107]
 [   159   1258]]

SVM Model Evaluation Metrics:
Accuracy: 0.9866559039426885
Precision: 0.434715821812596
Recall: 0.39943542695836276
F1 Score: 0.41632953291651337
Confusion Matrix:
 [[116776    736]
 [   851    566]]


In [ ]:
from xgboost import XGBClassifier
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix

# XGBoost Classifier
xgb_model = XGBClassifier(scale_pos_weight=sum(y_train_ros==0)/sum(y_train_ros==1),
                          use_label_encoder=False, eval_metric='logloss', random_state=42)
xgb_model.fit(X_train_ros, y_train_ros)
xgb_predictions = xgb_model.predict(X_test)


# Evaluating XGBoost Model
xgb_metrics = evaluate_model(y_test, xgb_predictions)

print("\nXGBoost Model Evaluation Metrics:")
print("Accuracy:", xgb_metrics[0])
print("Precision:", xgb_metrics[1])
print("Recall:", xgb_metrics[2])
print("F1 Score:", xgb_metrics[3])
print("Confusion Matrix:\n", xgb_metrics[4])



XGBoost Model Evaluation Metrics:
Accuracy: 0.9909778102901732
Precision: 0.5724515585509689
Recall: 0.9590684544812985
F1 Score: 0.716961223951464
Confusion Matrix:
 [[116497   1015]
 [    58   1359]]


In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import TensorDataset, DataLoader
from sklearn.preprocessing import StandardScaler

# Check if GPU is available and set the device accordingly
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Preprocess your data - Standardize features
scaler = StandardScaler()
X_train_ros_scaled = scaler.fit_transform(X_train_ros)
X_test_scaled = scaler.transform(X_test)

# Convert the data to PyTorch tensors
X_train_tensor = torch.tensor(X_train_ros_scaled, dtype=torch.float32)
y_train_tensor = torch.tensor(y_train_ros.values, dtype=torch.float32)
X_test_tensor = torch.tensor(X_test_scaled, dtype=torch.float32)
y_test_tensor = torch.tensor(y_test.values, dtype=torch.float32)

# Create datasets and dataloaders
train_dataset = TensorDataset(X_train_tensor, y_train_tensor)
train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True)

# Define the neural network architecture
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.fc1 = nn.Linear(X_train_ros.shape[1], 50)
        self.fc2 = nn.Linear(50, 20)
        self.fc3 = nn.Linear(20, 1)
        self.sigmoid = nn.Sigmoid()

    def forward(self, x):
        x = torch.relu(self.fc1(x))
        x = torch.relu(self.fc2(x))
        x = self.sigmoid(self.fc3(x))
        return x

# Initialize the network, loss function, and optimizer
model = Net().to(device)
criterion = nn.BCELoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

# Train the model
for epoch in range(10):  # loop over the dataset multiple times
    for i, (inputs, labels) in enumerate(train_loader):
        inputs, labels = inputs.to(device), labels.to(device)

        # Zero the parameter gradients
        optimizer.zero_grad()

        # Forward + backward + optimize
        outputs = model(inputs)
        loss = criterion(outputs, labels.unsqueeze(1))
        loss.backward()
        optimizer.step()

# Evaluate the model
model.eval()
with torch.no_grad():
    X_test_tensor = X_test_tensor.to(device)
    y_test_tensor = y_test_tensor.to(device)
    predictions = model(X_test_tensor)
    predictions = (predictions > 0.5).float()
    accuracy = (predictions.squeeze() == y_test_tensor).float().mean()
    print(f'Accuracy: {accuracy.item()}')



Accuracy: 0.9770283699035645


In [ ]:
from sklearn.metrics import precision_score, recall_score, f1_score, confusion_matrix

# Ensure the model is in evaluation mode
model.eval()

# Get predictions
with torch.no_grad():
    X_test_tensor = X_test_tensor.to(device)
    raw_predictions = model(X_test_tensor)
    predictions = (raw_predictions > 0.5).float().squeeze()  # Convert to binary predictions

# Move predictions and labels back to CPU for sklearn metrics
predictions = predictions.cpu().numpy()
y_test_np = y_test_tensor.cpu().numpy()

# Calculate additional metrics
precision = precision_score(y_test_np, predictions)
recall = recall_score(y_test_np, predictions)
f1 = f1_score(y_test_np, predictions)
conf_matrix = confusion_matrix(y_test_np, predictions)

# Print the metrics
print(f'Precision: {precision}')
print(f'Recall: {recall}')
print(f'F1 Score: {f1}')
print(f'Confusion Matrix:\n{conf_matrix}')


Precision: 0.33865030674846625
Recall: 0.9738884968242767
F1 Score: 0.5025491624180627
Confusion Matrix:
[[114817   2695]
 [    37   1380]]


In [ ]:
X_train_ros.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 939862 entries, 0 to 939861
Data columns (total 14 columns):
 #   Column                Non-Null Count   Dtype  
---  ------                --------------   -----  
 0   step                  939862 non-null  int64  
 1   customer              939862 non-null  int64  
 2   age                   939862 non-null  int64  
 3   gender                939862 non-null  int64  
 4   merchant              939862 non-null  int64  
 5   category              939862 non-null  int64  
 6   amount                939862 non-null  float64
 7   transaction_id        939862 non-null  int64  
 8   betweenness_customer  939862 non-null  float64
 9   pagerank_customer     939862 non-null  float64
 10  degree_customer       939862 non-null  int64  
 11  betweenness_merchant  939862 non-null  float64
 12  pagerank_merchant     939862 non-null  float64
 13  degree_merchant       939862 non-null  int64  
dtypes: float64(5), int64(9)
memory usage: 100.4 MB


In [ ]:
from sklearn.model_selection import GridSearchCV

param_grid = {
    'max_depth': [20],
    'learning_rate': [0.5],
    'n_estimators': [500, 1000, 300],
    'gamma': [0, 0.1, 0.2],
    'min_child_weight': [1, 5, 10],
    'subsample': [0.6, 0.8, 1.0],
    'colsample_bytree': [0.6, 0.8, 1.0],
}

grid_search = GridSearchCV(XGBClassifier(scale_pos_weight=sum(y_train_ros==0)/sum(y_train_ros==1),
                                         use_label_encoder=False, eval_metric='logloss', random_state=42),
                           param_grid, cv=3, scoring='accuracy', verbose=2)
grid_search.fit(X_train_ros, y_train_ros)


Fitting 3 folds for each of 9 candidates, totalling 27 fits
[CV] END gamma=0, learning_rate=0.5, max_depth=20, n_estimators=100; total time=   3.5s
[CV] END gamma=0, learning_rate=0.5, max_depth=20, n_estimators=100; total time=   3.6s
[CV] END gamma=0, learning_rate=0.5, max_depth=20, n_estimators=100; total time=   3.6s
[CV] END gamma=0, learning_rate=0.5, max_depth=20, n_estimators=200; total time=   6.9s
[CV] END gamma=0, learning_rate=0.5, max_depth=20, n_estimators=200; total time=   6.0s
[CV] END gamma=0, learning_rate=0.5, max_depth=20, n_estimators=200; total time=   7.3s
[CV] END gamma=0, learning_rate=0.5, max_depth=20, n_estimators=300; total time=   8.2s
[CV] END gamma=0, learning_rate=0.5, max_depth=20, n_estimators=300; total time=   9.8s
[CV] END gamma=0, learning_rate=0.5, max_depth=20, n_estimators=300; total time=   8.9s
[CV] END gamma=0.1, learning_rate=0.5, max_depth=20, n_estimators=100; total time=   2.6s
[CV] END gamma=0.1, learning_rate=0.5, max_depth=20, n_est

GridSearchCV(cv=3,
             estimator=XGBClassifier(base_score=None, booster=None,
                                     callbacks=None, colsample_bylevel=None,
                                     colsample_bynode=None,
                                     colsample_bytree=None, device=None,
                                     early_stopping_rounds=None,
                                     enable_categorical=False,
                                     eval_metric='logloss', feature_types=None,
                                     gamma=None, grow_policy=None,
                                     importance_type=None,
                                     interaction_constraints=None,
                                     learning_rate=...
                                     max_cat_to_onehot=None,
                                     max_delta_step=None, max_depth=None,
                                     max_leaves=None, min_child_weight=None,
                                     missing=nan, monotone_constraints=None,
                                     multi_strategy=None, n_estimators=None,
                                     n_jobs=None, num_parallel_tree=None,
                                     random_state=42, ...),
             param_grid={'gamma': [0, 0.1, 0.2], 'learning_rate': [0.5],
                         'max_depth': [20], 'n_estimators': [100, 200, 300]},
             scoring='accuracy', verbose=2)

In [ ]:
X_train_ros

,step,customer,age,gender,merchant,category,amount,transaction_id,betweenness_customer,pagerank_customer,degree_customer,betweenness_merchant,pagerank_merchant,degree_merchant
0,72,747,2,1,30,12,19.97,210964,1.017713e-08,0.000095,7,0.185934,0.038964,3931
1,153,1717,3,1,18,12,43.03,494706,2.307063e-09,0.000112,9,0.123490,0.035248,3576
2,17,3671,3,2,30,12,5.16,44111,6.309541e-08,0.000136,12,0.185934,0.038964,3931
3,173,3131,2,2,30,12,32.09,571605,1.060421e-08,0.000111,9,0.185934,0.038964,3931
4,77,1112,4,1,30,12,32.34,229916,1.377043e-08,0.000128,11,0.185934,0.038964,3931
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
939857,6,3274,5,1,43,7,306.04,15177,1.223375e-07,0.000186,18,0.094097,0.028834,2936
939858,91,3685,2,1,49,10,3.96,275626,1.697190e-06,0.000282,30,0.009831,0.007632,806
939859,25,2824,3,2,15,10,254.33,68271,2.014758e-07,0.000177,17,0.002795,0.004193,444
939860,51,1005,4,2,32,8,183.00,144725,1.113953e-06,0.000232,24,0.000856,0.002253,244


In [ ]:
best_parameters =  gridsearch.best_parameters
xgb_model_optimized = XGBClassifier(**best_parameters, use_label_encoder=False, eval_metric='logloss', random_state=42)
xgb_model_optimized.fit(X_train_ros, y_train_ros)
optimized_predictions = xgb_model_optimized.predict(X_test)
optimized_metrics = evaluate_model(y_test, optimized_predictions)

print("\nOptimized XGBoost Model Evaluation Metrics:")
print("Accuracy:", optimized_metrics[0])
print("Precision:", optimized_metrics[1])
print("Recall:", optimized_metrics[2])
print("F1 Score:", optimized_metrics[3])
print("Confusion Matrix:\n", optimized_metrics[4])


Best Parameters: {'gamma': 0, 'learning_rate': 0.5, 'max_depth': 20, 'n_estimators': 300}
Best Score: 0.9986944891841709

Optimized XGBoost Model Evaluation Metrics:
Accuracy: 0.9964937063289862
Precision: 0.8448275862068966
Recall: 0.8645024700070572
F1 Score: 0.8545517963027555
Confusion Matrix:
 [[117287    225]
 [   192   1225]]


In [ ]:
best_parameters =  {'gamma': 0, 'learning_rate': 0.5, 'max_depth': 20, 'n_estimators': 300}

xgb_model_optimized = XGBClassifier(**best_parameters, use_label_encoder=False, eval_metric='logloss', random_state=42)
xgb_model_optimized.fit(X_train_ros, y_train_ros)
optimized_predictions = xgb_model_optimized.predict(X_test)
optimized_metrics = evaluate_model(y_test, optimized_predictions)

print("\nOptimized XGBoost Model Evaluation Metrics:")
print("Accuracy:", optimized_metrics[0])
print("Precision:", optimized_metrics[1])
print("Recall:", optimized_metrics[2])
print("F1 Score:", optimized_metrics[3])
print("Confusion Matrix:\n", optimized_metrics[4])


<class 'pandas.core.frame.DataFrame'>
Int64Index: 118929 entries, 70803 to 450107
Data columns (total 14 columns):
 #   Column                Non-Null Count   Dtype  
---  ------                --------------   -----  
 0   step                  118929 non-null  int64  
 1   customer              118929 non-null  int64  
 2   age                   118929 non-null  int64  
 3   gender                118929 non-null  int64  
 4   merchant              118929 non-null  int64  
 5   category              118929 non-null  int64  
 6   amount                118929 non-null  float64
 7   transaction_id        118929 non-null  int64  
 8   betweenness_customer  118929 non-null  float64
 9   pagerank_customer     118929 non-null  float64
 10  degree_customer       118929 non-null  int64  
 11  betweenness_merchant  118929 non-null  float64
 12  pagerank_merchant     118929 non-null  float64
 13  degree_merchant       118929 non-null  int64  
dtypes: float64(5), int64(9)
memory usage: 13.6 MB


In [ ]:
X_train_ros

,step,customer,age,gender,merchant,category,amount,transaction_id,betweenness_customer,pagerank_customer,degree_customer,betweenness_merchant,pagerank_merchant,degree_merchant
0,72,747,2,1,30,12,19.97,210964,1.017713e-08,0.000095,7,0.185934,0.038964,3931
1,153,1717,3,1,18,12,43.03,494706,2.307063e-09,0.000112,9,0.123490,0.035248,3576
2,17,3671,3,2,30,12,5.16,44111,6.309541e-08,0.000136,12,0.185934,0.038964,3931
3,173,3131,2,2,30,12,32.09,571605,1.060421e-08,0.000111,9,0.185934,0.038964,3931
4,77,1112,4,1,30,12,32.34,229916,1.377043e-08,0.000128,11,0.185934,0.038964,3931
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
939857,6,3274,5,1,43,7,306.04,15177,1.223375e-07,0.000186,18,0.094097,0.028834,2936
939858,91,3685,2,1,49,10,3.96,275626,1.697190e-06,0.000282,30,0.009831,0.007632,806
939859,25,2824,3,2,15,10,254.33,68271,2.014758e-07,0.000177,17,0.002795,0.004193,444
939860,51,1005,4,2,32,8,183.00,144725,1.113953e-06,0.000232,24,0.000856,0.002253,244


In [ ]:
X_train.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 475714 entries, 210963 to 121958
Data columns (total 14 columns):
 #   Column                Non-Null Count   Dtype  
---  ------                --------------   -----  
 0   step                  475714 non-null  int64  
 1   customer              475714 non-null  int64  
 2   age                   475714 non-null  int64  
 3   gender                475714 non-null  int64  
 4   merchant              475714 non-null  int64  
 5   category              475714 non-null  int64  
 6   amount                475714 non-null  float64
 7   transaction_id        475714 non-null  int64  
 8   betweenness_customer  475714 non-null  float64
 9   pagerank_customer     475714 non-null  float64
 10  degree_customer       475714 non-null  int64  
 11  betweenness_merchant  475714 non-null  float64
 12  pagerank_merchant     475714 non-null  float64
 13  degree_merchant       475714 non-null  int64  
dtypes: float64(5), int64(9)
memory usage: 54.4 MB
